# Builder (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"

#r @"../../../../../../../.nuget/packages/asyncio/0.1.69/lib/netstandard2.0/AsyncIO.dll"
#r @"../../../../../../../.nuget/packages/netmq/4.0.1.12/lib/netstandard2.1/NetMQ.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem

In [ ]:
let build path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let dir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let extension = fullPath |> System.IO.Path.GetExtension

    let getLocals () = $"dir: {dir} / fileName: {fileName} / extension: {extension} / {getLocals ()}"
    trace Debug (fun () -> "build") getLocals

    let targetPath =
        dir </> "target"

    let fsprojPath =
        targetPath </> $"{fileName}.fsproj"

    let repoRoot =
        let rec loop dir =
            if dir </> ".paket" |> System.IO.Directory.Exists
            then dir
            else dir |> System.IO.Directory.GetParent |> fun x -> x.FullName |> loop
        loop dir

    let fsprojCode = $"""<Project Sdk="Microsoft.NET.Sdk">
    <PropertyGroup>
        <TargetFramework>net8.0</TargetFramework>
        <LangVersion>preview</LangVersion>
        <RollForward>Major</RollForward>
        <TargetLatestRuntimePatch>true</TargetLatestRuntimePatch>
        <Version>0.0.1-alpha.1</Version>
        <OutputType>Exe</OutputType>
    </PropertyGroup>

    <ItemGroup>
        <Compile Include="{repoRoot}/nbs/Common.fs" />
        <Compile Include="{repoRoot}/nbs/Async.fs" />
        <Compile Include="{repoRoot}/nbs/AsyncSeq.fs" />
        <Compile Include="{repoRoot}/nbs/Runtime.fs" />
        <Compile Include="{repoRoot}/nbs/FileSystem.fs" />
        <Compile Include="{fullPath}" />
    </ItemGroup>

    <Import Project="{repoRoot}/.paket/Paket.Restore.targets" />
</Project>
"""
    System.IO.Directory.CreateDirectory targetPath |> ignore

    let paketReferencesCode = $"FSharp.Core

System.Reactive.Linq
FSharp.Control.AsyncSeq
FParsec
"

    do! System.IO.File.WriteAllTextAsync (targetPath </> "paket.references", paketReferencesCode) |> Async.AwaitTask

    do! System.IO.File.WriteAllTextAsync (fsprojPath, fsprojCode) |> Async.AwaitTask

    let! exitCode, _result =
        Runtime.executeWithOptionsAsync
            {
                Command = "dotnet build -c Release"
                CancellationToken = None
                OnLine = None
                WorkingDirectory = Some targetPath
            }

    return exitCode
}

In [ ]:

let main args =
    let paths =
        match args |> Array.tryHead |> Option.defaultValue "" with
        | "" | null -> [||]
        | path when System.IO.File.Exists path -> [| path |]
        | path when path |> String.contains ";" -> path |> String.split [| ';' |]
        | _ -> [||]

    paths
    |> Array.map build
    |> Async.Parallel
    |> Async.RunSynchronously
    |> Array.sum

In [ ]:
//// test

main [| System.Environment.GetEnvironmentVariable "OUTPUT" |]

0

00:27:52 #1 [Debug] build / dir: C:\home\git\polyglot\apps\builder / fileName: Builder / extension: .fs
00:27:52 #2 [Debug] executeAsync / options: { Command = "dotnet build -c Release"
  WorkingDirectory = Some "C:\home\git\polyglot\apps\builder\target"
  CancellationToken = None
  OnLine = None }
00:27:53 #3 [Debug]  29508: MSBuild version 17.7.0+5785ed5c2 for .NET
00:27:54 #4 [Debug]  29508:   Determining projects to restore...
00:27:55 #5 [Debug]  29508:   All projects are up-to-date for restore.
00:27:55 #6 [Debug]  29508: C:\Users\i574n\scoop\apps\dotnet-sdk-preview\current\sdk\8.0.100-preview.6.23330.14\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference.targets(314,5): message NETSDK1057: You are using a preview version of .NET. See: https://aka.ms/dotnet-support-policy [C:\home\git\polyglot\apps\builder\target\Builder.fsproj]
00:28:02 #7 [Debug]  29508:   Builder -> C:\home\git\polyglot\apps\builder\target\bin\Release\net8.0\Builder.dll
00:28:02 #8 [Debug] 